In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

2025-03-21 18:22:43.881031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742548963.972517  301109 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742548963.998441  301109 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742548964.223727  301109 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742548964.223752  301109 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742548964.223754  301109 computation_placer.cc:177] computation placer alr

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/'

SEQUENCE_LENGTH = 30

sequences = []
labels = []

landmark_points = [ 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ]

for label_folder in os.listdir(DATASET_DIR):
    if label_folder == "standing": label = 0
    elif label_folder == "pushup": label = 1
    elif label_folder == "stand_knee_raise": label = 2
    elif label_folder == "stand_shoulder_press": label = 3

    label_path = os.path.join(DATASET_DIR, label_folder)
    if not os.path.isdir(label_path):
        continue
    for video_file in os.listdir(label_path):
        video_path = os.path.join(label_path, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []
                neck_x = (results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) / 2
                neck_y = (results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) / 2
                for idx, lm in enumerate(results.pose_landmarks.landmark):
                    if idx in landmark_points:
                        x = lm.x - neck_x
                        y = lm.y - neck_y
                        landmarks.extend([x, y])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
                    labels.append(label)
        cap.release()

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_categorical = to_categorical(labels)

In [ ]:
X = np.array(sequences)
y = np.array(labels_categorical)

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
early_stopping = EarlyStopping(
    monitor='loss',      # validation loss를 모니터링
    patience=10,         # 10 epoch 동안 개선 없으면 멈춤
    restore_best_weights=True # 가장 좋은 가중치 복원
)

model = Sequential()
model.add(LSTM(64, return_sequences=False, input_shape=(SEQUENCE_LENGTH, 24)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=100, callbacks=[early_stopping], validation_data=(X_val, y_val))

In [ ]:
model.save('exercise_classifier.h5')